# Bert NER on SageMaker using PyTorch

This uses the Biocreative II gene mention dataset https://biocreative.bioinformatics.udel.edu/tasks/biocreative-ii/task-1a-gene-mention-tagging/






In [1]:
import sys, os
import logging

sys.path.append("src")

logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Bucket and role set up

In [2]:
import sagemaker
from sagemaker import get_execution_role
sm_session = sagemaker.session.Session()
role ="arn:aws:iam::324346001917:role/service-role/AmazonSageMaker-ExecutionRole-20181222T162635" #get_execution_role()


2020-10-01 09:07:14,965 - sagemaker.analytics - WARNING - pandas failed to import. Analytics features will be impaired or broken.
2020-10-01 09:07:15,265 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
data_bucket = "aegovan-data"

data_bucket_prefix = "Biocreative-gene-mention"

s3_uri_data = "s3://{}/{}/train data".format(data_bucket, data_bucket_prefix)
s3_uri_train = "{}/{}".format(s3_uri_data, "train.in")
s3_uri_classes = "{}/{}".format(s3_uri_data, "GENE.eval")

s3_uri_test = "s3://{}/{}/test data".format(data_bucket, data_bucket_prefix , "test.in")

s3_output_path = "s3://{}/{}/output".format(data_bucket, data_bucket_prefix)
s3_code_path = "s3://{}/{}/code".format(data_bucket, data_bucket_prefix)
s3_checkpoint = "s3://{}/{}/checkpoint".format(data_bucket, data_bucket_prefix)

## Train

This shows you how to train BERT on SageMaker using SPOT instances

In [4]:
inputs_full =  {
    "train" : s3_uri_train,
    "class" : s3_uri_classes
}

inputs = inputs_full

In [5]:
sm_localcheckpoint_dir="/opt/ml/checkpoints/"

In [6]:
instance_type = "ml.p3.8xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [7]:
hp = {
"epochs" : 10,
"earlystoppingpatience" : 3,
# Increasing batch size might end up with CUDA OOM error, increase grad accumulation instead
"batch" : 8 * instance_type_gpu_map[instance_type],
"trainfile" :s3_uri_train.split("/")[-1],
"classfile":s3_uri_classes.split("/")[-1],
# The number of steps to accumulate gradients for
"gradaccumulation" : 4,
"log-level":"INFO",
# This param depends on your model max pos embedding size or when large you might end up with CUDA OOM error    
"maxseqlen" : 512,
# Make sure the lr is quite small, as this is a pretrained model..
"lr":0.00001,
# Use finetuning (set to 1), if you only want to change the weights in the final classification layer.. 
"finetune": 0,
"checkpointdir" : sm_localcheckpoint_dir,
# Checkpoints once every n epochs
"checkpointfreq": 2
}



In [8]:
hp

{'epochs': 10,
 'earlystoppingpatience': 3,
 'batch': 32,
 'trainfile': 'train.in',
 'classfile': 'GENE.eval',
 'gradaccumulation': 4,
 'log-level': 'INFO',
 'maxseqlen': 512,
 'lr': 1e-05,
 'finetune': 0,
 'checkpointdir': '/opt/ml/checkpoints/',
 'checkpointfreq': 2}

In [9]:
inputs

{'train': 's3://aegovan-data/Biocreative-gene-mention/train data/train.in',
 'class': 's3://aegovan-data/Biocreative-gene-mention/train data/GENE.eval'}

In [10]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ]

In [11]:
# set True if you need spot instance
use_spot = True
train_max_run_secs =   2*24 * 60 * 60
spot_wait_sec =  5 * 60
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    # Use smaller dataset to run locally
    inputs = inputs_sample


In [12]:
job_type = "bc2-ner-bert"
base_name = "{}".format(job_type)

In [13]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='main.py',
                    source_dir = 'src',
                    role=role,
                    framework_version ="1.4.0",
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters = hp,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    volume_size=30,
                    code_location=s3_code_path,
                    debugger_hook_config=False,
                    base_job_name =base_name,  
                    use_spot_instances = use_spot,
                    max_run =  train_max_run_secs,
                    max_wait = max_wait_time_secs,   
                    checkpoint_s3_uri=s3_checkpoint,
                    checkpoint_local_path=sm_localcheckpoint_dir)

estimator.fit(inputs, wait=True)

2020-10-01 09:07:15,567 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2020-10-01 09:07:17,707 - sagemaker - INFO - Creating training-job with name: bc2-ner-bert-2020-09-30-23-07-15-632
2020-09-30 23:07:18 Starting - Starting the training job...
2020-09-30 23:07:20 Starting - Launching requested ML instances.........
2020-09-30 23:09:14 Starting - Preparing the instances for training...
2020-09-30 23:09:56 Downloading - Downloading input data...
2020-09-30 23:10:21 Training - Downloading the training image......
2020-09-30 23:11:54 Uploading - Uploading generated training model
2020-09-30 23:11:54 Failed - Training job failed
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-09-30 23:11:37,495 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-09-30 23:11:37,537 sagemaker_pytorch_container.training INFO     Block until all host DNS

{'trainfile': 'train.in', 'traindir': '/opt/ml/input/data/train', 'classfile': 'GENE.eval', 'classdir': '/opt/ml/input/data/class', 'outdir': '/opt/ml/output/data', 'modeldir': '/opt/ml/model', 'checkpointdir': '/opt/ml/checkpoints/', 'checkpointfreq': '2', 'earlystoppingpatience': 3, 'epochs': 10, 'gradaccumulation': 4, 'batch': 32, 'lr': 1e-05, 'finetune': 0, 'maxseqlen': 512, 'log_level': 'INFO'}
2020-09-30 23:11:51,504 sagemaker-containers ERROR    ExecuteUserScriptError:
Command "/opt/conda/bin/python main.py --batch 32 --checkpointdir /opt/ml/checkpoints/ --checkpointfreq 2 --classfile GENE.eval --earlystoppingpatience 3 --epochs 10 --finetune 0 --gradaccumulation 4 --log-level INFO --lr 1e-05 --maxseqlen 512 --trainfile train.in"
Traceback (most recent call last):
  File "main.py", line 80, in <module>
    main()
  File "main.py", line 56, in main
    learning_rate=args.lr, fine_tune=args.finetune, model_dir=args.modeldir)
  File "/opt/ml/code/builder.py", line 38, in __init__
 

UnexpectedStatusException: Error for Training job bc2-ner-bert-2020-09-30-23-07-15-632: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python main.py --batch 32 --checkpointdir /opt/ml/checkpoints/ --checkpointfreq 2 --classfile GENE.eval --earlystoppingpatience 3 --epochs 10 --finetune 0 --gradaccumulation 4 --log-level INFO --lr 1e-05 --maxseqlen 512 --trainfile train.in"
Traceback (most recent call last):
  File "main.py", line 80, in <module>
    main()
  File "main.py", line 56, in main
    learning_rate=args.lr, fine_tune=args.finetune, model_dir=args.modeldir)
  File "/opt/ml/code/builder.py", line 38, in __init__
    if self.num_workers <= 0:
AttributeError: 'Builder' object has no attribute 'num_workers'

## Deploy BERT model

#### Inference container
Ideally the server containing should already have all the required dependencies installed to reduce start up time and ensure that the runtime enviornment is consistent. This can be implemented using a custom docker image.

But for this demo, to simplify, we will let the Pytorch container script model install the dependencies during start up. As a result, you will see some of the initial ping requests fail, until all dependencies are installed.


In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
role = get_execution_role()

model_uri = estimator.model_data

model = PyTorchModel(model_data=model_uri,
                     role=role,
                     framework_version='1.4.0',
                     entry_point='serve.py',
                     source_dir='src')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

### Invoke API

In [ ]:
data = ["Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice",
        "ET is a sci-fi directed by steven spielberg"]

In [ ]:
from sagemaker.content_types import CONTENT_TYPE_CSV
import json

predictor.content_type = CONTENT_TYPE_CSV
predictor.serializer=None
predictor.deserializer = None


data_bytes="\n".join(data).encode("utf-8")
response_bytes  = predictor.predict(data_bytes,  
                                    initial_args={ "Accept":"text/json", "ContentType" : CONTENT_TYPE_CSV }
                                   )

json.loads(response_bytes.decode("utf-8"))

## Delete endpoint

In [ ]:
predictor.delete_endpoint()